# 회귀(Regression)

예측할 값(Target)이 연속형(continuous) 데이터(float)인 지도 학습(Supervised Learning).

## 회귀의 주요 평가 지표

- ### MSE (Mean Squared Error)
    - 실제 값과 예측값의 차를 제곱해 평균 낸 것
    - scikit-learn 평가함수: mean_squared_error() 
    - 교차검증시 지정할 문자열: 'neg_mean_squared_error'
    $$
    MSE = \frac{1}{n}\sum_{i=1}^{n}(y_i - \hat{y_i})^2\\
    y_i: 실제값, \hat{y_i}: 모델이 예측한 값
    $$
    

- ### RMSE (Root Mean Squared Error)
    - MSE는 오차의 제곱한 값이므로 실제 오차의 평균보다 큰 값이 나온다.  MSE의 제곱근이 RMSE이다.
    - mean_squared_error() 의 squared=False로 설정해서 계산. 또는 MSE를 구한 뒤 np.sqrt()로 제곱근을 구한다.
    - 교차검증시 지정할 문자열: 'neg_root_mean_squared_error'
    
    $$
    RMSE = \sqrt{\frac{1}{n}\sum_{i=1}^{n}(y_i - \hat{y_i})^2}
    $$

- ### $R^2$ (R square, 결정계수)
    - 평균으로 예측했을 때 오차(총오차) 보다 모델을 사용했을 때 얼마 만큼 더 좋은 성능을 내는지를 비율로 나타낸 값. 
    - 1에 가까울 수록 좋은 모델.
    - scikit-learn 평가함수: r2_score()
    - 교차검증시 지정할 문자열: 'r2'
    - [참고](https://ko.khanacademy.org/math/statistics-probability/describing-relationships-quantitative-data/assessing-the-fit-in-least-squares-regression/a/r-squared-intuition)
    $$
    R^2 = \cfrac{\sum_{i=1}^{n}(\hat{y_i}-\bar{y})^2}{\sum_{i=1}^{n}(y_i - \bar{y})^2}\\
    R^2 = 1 - \cfrac{\sum_{i=1}^{n}(y_i - \hat{y_i})^2}{\sum_{i=1}^{n}(y_i - \bar{y})^2}
    $$

- $y_i$ : i번째 정답 값, 
- $\hat{y_i}$ : i 번째 예측 값, 
- $\bar{y}$ : y의 평균    

### 예제

##### import

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import make_regression

##### Dataset 생성
- make_xxxxx() 함수
    - 머신러닝 학습을 위한 dummy dataset 구현 함수
    - 필요한 설정을 직접하여 테스트할 수 있는 데이터셋을 생성해준다.
- make_regression(): 회귀 문제를 위한 dummy dataset 생성
- make_classification(): 분류 문제를 위한 dummy dataset 생성

In [ ]:
X, y = make_regression(n_samples=1000,  # 데이터 개수
                       n_features=1,  # feature(컬럼, X)의 개수
                       n_informative=1,   # label(target, y)에 영향을 주는 feature 개수. feature개수보다 작거나 같은 정수지정.
                       noise=30,  # 잡음->인정할 수 있는 오차범위. 모델이 찾을 수없는 값의 범위지정. 모델이 만든 값에 0 ~ 30 범위의 난수를 더한다.
                       random_state=0
                      )

In [ ]:
X.shape, y.shape

In [ ]:
print(X[0])
print(y[0])

> #### Noise란 
>  같은 Feature를 가진 데이터포인트가 다른 label을 가지는 이유를 Noise(노이즈)라고 한다. 단 그 이유는 현재 상태에선 모른다. 예를 들어 나이란 Feature가 있고 구매량이란 target이 있을때 같은 나이인데 구매량이 다른 경우 그 이유를 우리는 알 수 없다. 그 차이를 만드는 나이 이외의 Feature가 있는데 그것이 수집이 되지 않은 것이다.  그래서 데이터 수집하고 전처리 할 때 그 이유가 되는 Feature를 찾아야 한다. 찾으면 성능이 올라가는 것이고 못찾으면 모르는 이유가 되어 모델 성능이 떨어진다. 

In [ ]:
plt.scatter(X, y, alpha=0.3)
plt.show()

In [ ]:
# 상관계수
print("x, y의 상관계수\n", np.corrcoef(X.flatten(), y))
print('y평균:', np.mean(y), "y중앙값", np.median(y), "y최소/최대값", np.min(y), np.max(y))

##### 모델 생성, 학습

In [ ]:
from sklearn.linear_model import LinearRegression  
# 선형회귀 모델. X와 y간에 상관성이 높을때 사용하며 직선의 방정식을 이용해 추론모델을 만든다.

lr = LinearRegression()   # 기울기 * X + 절편  => 기울기와 절편을 학습시 찾는다.
lr.fit(X, y)
pred = lr.predict(X)  
print(pred[:5])
print(y[:5])

##### 평가

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

print("MSE:", mean_squared_error(y, pred))  
print("RMSE:", np.sqrt(mean_squared_error(y, pred)), mean_squared_error(y, pred, squared=False))
print("R2:", r2_score(y, pred))


##### 교차검증 (cross validation)

In [ ]:
from sklearn.model_selection import cross_val_score
# cross_val_score(), GridSearchCV(), RandomizedSearchCV()
score = cross_val_score(LinearRegression(), # 모델
                        X, #Feature
                        y, #target, 
                        scoring="neg_mean_squared_error",   # -1 * MSE
                        cv=5, #나눌 폴드 개수
                        n_jobs=-1
                       )

In [ ]:
print(-1 * score)
np.mean(-1*score)

##### 모델이 찾은 계수(coef, 가중치-weigth)와 절편(intercept, 편향-bias) 조회
- LinearRegression 모델이 학습해서 찾는 파라미터 제공 attribute
    - coef_: Feature에 곱하는 가중치
    - intercept_: 모든 Feature가 0일때 예측값

In [ ]:
print('기울기(coef, 가중치):', lr.coef_)
print('절편(intercept, bias(편향)):', lr.intercept_)

In [ ]:
print(pred[:5].flatten() )
print(pred[:5])

In [ ]:
y[:5]

In [ ]:
X[:5]

##### X, y와 추론결과  시각화

In [ ]:
plt.scatter(X, y, alpha=0.3)  # X와 정답간의 관계
y_hat = lr.predict(X)  # X*lr.coef_ + lr.intercept_  # X와 모델이 추론한값
plt.plot(X, y_hat, color='red')

plt.show()

## 기존 분류 모델의 회귀 모델

##### import

In [ ]:
# XXXXXClassifier: 분류,  XXXXXRegressor : 회귀
from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# 회귀 데이터셋: stratify를 지정하지 않는다.
X_train.shape, X_test.shape

##### 모델들 생성

In [ ]:
knn = KNeighborsRegressor(n_neighbors=3)
tree = DecisionTreeRegressor(max_depth=3, random_state=0)
rfr = RandomForestRegressor(n_estimators=200, max_depth=2)
gb = GradientBoostingRegressor(n_estimators=200, max_depth=2)
lr = LinearRegression()
xgb = XGBRegressor(n_estimators=200, max_depth=2, learning_rate=0.01)

estimators = [
    ('knn', knn),
    ('Tree', tree),
    ('Random Forest', rfr),
    ('gradient boosting', gb),
    ('Linear Regression', lr),
    ('XGBoost', xgb)
]

##### 평가출력 함수

In [ ]:
%%writefile metrics.py
# %load metrics.py

from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import precision_recall_curve, average_precision_score, PrecisionRecallDisplay, roc_curve, roc_auc_score, RocCurveDisplay
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

# 회귀문제 평가함수
def print_metrics_regression(y, pred, title=None):
    """
    회귀문제 평가지표를 출력하는 함수
    MSE, RMSE, R2 세개의 평가지표를 출력
    [parameter]
        y: ndarray - 정답(target)
        pred: ndarray - 모델이 추론한 결과
        title: str - 출력할 내용의 title. 기본값: None = 출력안한다.
    [return value]
    [exception]
    """
    if title:
        print(title)
    mse = mean_squared_error(y, pred)
    rmse = mean_squared_error(y, pred, squared=False)
    r2 = r2_score(y, pred)
    print(f'MSE: {mse}, RMSE: {rmse}, R2: {r2}')

# 분류문제 평가 함수
def print_metrics_classification(y, pred, title=None):
    """
    정답(target)과 모델이 추론한 값을 받아 정확도, 재현율, 정밀도, f1 점수를 출력
    [parameter]
        y: ndarray - 정답(target)
        pred: ndarray - 모델이 추론한 결과
        title: str - 출력할 내용의 title. 기본값: None = 출력안한다.
    [return value]
    [exception]
    """
    if title:
        print(title)
    print(f'정확도(accuracy): {accuracy_score(y, pred)}, 재현율(recall):{recall_score(y, pred)}, 정밀도(precision):{precision_score(y, pred)}, F1점수:{f1_score(y, pred)}')

def plot_confusionmatrix(y, pred, title=None):
    """
    정답과 모델이 추론한 결과를 이용해 Confusion Matrix를 시각화(히트맵)하는 함수
    [parameter]
        y: ndarray - 정답(target)
        pred: ndarray - 모델이 추론한 결과
        title: str - 출력할 내용의 title. 기본값: None = 출력안한다.
    """
    cm = confusion_matrix(y, pred)
    disp = ConfusionMatrixDisplay(cm)
    disp.plot(cmap='Blues')
    if title:
        plt.title(title)
    plt.show()

    
def plot_precisionrecall_curve(y, pos_proba, title=None):
    """
    Precision Recall Curve 를 시각화하는 함수
    [parameter]
        y: ndarray - 정답
        pos_proba: ndarray - positive(양성)의 확률
        title: str - 출력할 내용의 title
    """
    ap_score = average_precision_score(y, pos_proba)
    precisions, recalls, threshs = precision_recall_curve(y, pos_proba)
    disp = PrecisionRecallDisplay(precisions, recalls, average_precision=ap_score)
    disp.plot()
    if title:
        plt.title(title)
    plt.show()
    
def plot_roccurve(y, pos_proba, title=None):
    """
    ROC Curve를 시각화하는 함수
    [parameter]
        y: ndarry - 정답
        pos_proba: ndarray - positive(양성)의 확률
        title: str - 출력할 내용의 title
    """
    auc_score = roc_auc_score(y, pos_proba)
    fprs, tprs, threshs = roc_curve(y, pos_proba)
    disp = RocCurveDisplay(fpr=fprs, tpr=tprs, roc_auc=auc_score)
    disp.plot()
    if title:
        plt.title(title)
    plt.show()


In [ ]:
from metrics import print_metrics_regression

In [ ]:
print_metrics_regression(y, pred)

##### 모델 학습 및 평가

In [ ]:
for model_name, model in estimators:
#     학습
    model.fit(X_train, y_train)
#     추론
    pred_train = model.predict(X_train)
    pred_test = model.predict(X_test)
#     평가
    print_metrics_regression(y_train, pred_train, f"{model_name}의 Train set 평가")
    print_metrics_regression(y_test, pred_test, f"{model_name}의 Test set 평가")
    print("="*50)

##### Voting
- VotingRegressor 
    - 각 모델이 예측한 값의 평균을 출력한다.

In [ ]:
voting = VotingRegressor(estimators=estimators)
voting.fit(X_train, y_train)

In [ ]:
pred_train2 = voting.predict(X_train)
pred_test2 = voting.predict(X_test)

In [ ]:
print_metrics_regression(y_train, pred_train2)

In [ ]:
print_metrics_regression(y_test, pred_test2)

##### DecisionTreeRegressor Tree 시각화

In [ ]:
from sklearn.tree import export_graphviz
from graphviz import Source

graph = Source(export_graphviz(tree, 
#                                feature_names=xxx,
                               rounded=True, 
                               filled=True                          
                              ))
graph